<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9A%D0%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Конечные автоматы Мили и Мура
Вариант 5

In [1]:
import numpy as np
import pandas as pd
import random
import copy
from prettytable import PrettyTable
import itertools as it

In [2]:
def equiv(a,b): # эквивалентность ~
  rez = 0
  if((a == 1 and b == 1) or (a == 0 and b == 0)): rez = 1
  return rez

def implic(a,b): # импликация ->
  rez = 1
  if(a == 1 and b == 0): rez = 0
  return rez

def pier(a,b): # стрелка Пирса ^
  rez = 0
  if(a == 0 and b == 0): rez = 1
  return rez

def schaeffer(a,b): # штрих Шеффера|
  rez = 1
  if(a == 1 and b == 1): rez = 0
  return rez

#Очерёдность выполнения логических операций:
#1-Инверсия;
#2-Конъюнкция и штрих Шеффера;
#3-Дизъюнкция и стрелка Пирса;
#4-Импликация и сложение по модулю 2;
#5-Эквивалентность

##Построение таблицы истинности:

In [3]:
#таблица истинности
def truthTable(p_inp, p_st, x, v):
  inner_table = [[(str(x1(p_st[i], p_inp[j])) + str(x2(p_st[i], p_inp[j]))) for j in range(len(p_inp))] for i in range(len(p_st))]

  print('Таблица истинности')
  table = PrettyTable()
  columns = p_st.copy()
  columns.insert(0, "v \ x")
  table.field_names = columns

  for i in range(len(p_inp)):
    table.add_row([p_inp[i], inner_table[i][0], inner_table[i][1],
                             inner_table[i][2], inner_table[i][3]])

  columns.clear()
  print(table)
  return inner_table

#----------------------------------------------------------------------------------------------------

#таблица выходов для автомата Мили (выход зависит только от состояния)
def outputTable_Mili(p_inp, p_st, x, v):
  inner_table = [(str(y1(p_st[j], p_inp[j])) + str(y2(p_st[j], p_inp[j]))
                  + str(y3(p_st[j], p_inp[j])) + str(y4(p_st[j], p_inp[j]))) for j in range(len(p_inp))]

  print('Таблица выходов')
  table = PrettyTable()
  columns = p_st.copy()
  columns.insert(0, "y \ x")
  table.field_names = columns

  table.add_row(['y', inner_table[0], inner_table[1], inner_table[2], inner_table[3]])

  columns.clear()
  print(table)
  return inner_table

#таблица выходов для автомата Мура (выход зависит и от состояния, и от входа)
def outputTable_Mura(p_inp, p_st, x, v):
  inner_table = [[(str(y1(p_st[i], p_inp[j])) + str(y2(p_st[i], p_inp[j]))
                  + str(y3(p_st[i], p_inp[j])) + str(y4(p_st[i], p_inp[j]))) for j in range(len(p_inp))] for i in range(len(p_st))]

  print('Таблица выходов')
  table = PrettyTable()
  columns = p_st.copy()
  columns.insert(0, "v \ x")
  table.field_names = columns

  for i in range(len(p_inp)):
    table.add_row([p_inp[i], inner_table[i][0], inner_table[i][1],
                             inner_table[i][2], inner_table[i][3]])

  columns.clear()
  print(table)
  return inner_table

#----------------------------------------------------------------------------------------------------

#матрица соединений
def creatly_connection_table(trans, out):
  table = pd.DataFrame('0', index=range(trans.shape[1]-1), columns=trans.columns)

  if(len(out) != 1):
    for i in range(table.shape[0]):
      table['input'][i] = table.columns[i]

  for i in table.columns[0:table.columns.shape[0]-1]:
    for j in range(table.shape[0]):
      cell = trans.index[trans[i] == table[table.columns[table.shape[1]-1]][j]]
      if len(cell) != 0:
        table[i][j] = '';
        for k in range(len(cell)):
          table[i][j] += out[out.columns[out.shape[1]-1]][cell[k]] + '/' + out[i][cell[k]]
          if k != len(cell)-1:
            table[i][j] += ' v '

  return table

###Задача 1. Мили

In [4]:
x0 = [0, 1]

#2 входа - v
v = [0, 1]
#2 состояния - x1/x2
x = [0, 1]
#4 выхода - y1/y2/y3/y4

#+(сумма по модулю 22), ^(стрелка пирса), & и v(конъюнкция и дизъюнкция)
#|(штрих шеффера), '(отрицание)
#->(импликация) и ~(эквивалентность)

In [5]:
pos_inputs = []
for i in range(len(v)):
  for j in range(len(v)):
    pos_inputs.append(str(v[i]) + str(v[j]))
print('Возможные входы:', pos_inputs)

pos_states = []
for i in range(len(x)):
  for j in range(len(x)):
    pos_states.append(str(x[i]) + str(x[j]))
print('Возможные состояния:', pos_states)

Возможные входы: ['00', '01', '10', '11']
Возможные состояния: ['00', '01', '10', '11']


In [6]:
#Выходы и состояния для автомата Мили заданы функциями с логическими операторами

def x1(x, v):
  #v1 + (x2 | x1 + (v2 v (v2 + (x2 ^ x2))))

  rezult = int(v[0]) ^ (schaeffer(int(x[1]), int(x[0])) ^ (int(v[1]) or (int(v[1]) ^ pier(int(x[1]), int(x[1])))))
  return rezult

def x2(x, v):
  #v1 -> (v1 | (x1' | v1 + (x1 ^ v2)))

  rezult = implic(int(v[0]), schaeffer(int(v[0]), schaeffer(not int(x[0]), int(v[0])) ^ pier(int(x[0]), int(v[1]))))
  return rezult

def y1(x, v):
  #(x1 v x2) + (x2 | x1) ~ x1 ~ (x2' -> x1)

  rezult = equiv(equiv(int(x[0]) or int(x[1]) ^ schaeffer(int(x[1]), int(x[0])), int(x[0])), implic(not int(x[1]), int(x[0])))
  return rezult

def y2(x, v):
  #x1 ~ x2 ^ (x1 + x2 | (x2 ~ x2))

  rezult = equiv(int(x[0]), pier(int(x[1]), int(x[0]) ^ schaeffer(int(x[1]), equiv(int(x[1]), int(x[1])))))
  return rezult

def y3(x, v):
  #x2 & x2 | x2 + x2 ^ x2

  rezult = schaeffer(int(x[1]) and int(x[1]), int(x[1])) ^ pier(int(x[1]), int(x[1]))
  return rezult

def y4(x, v):
  #x1 v x1 ~ x2 & (x2 ^ x1 ^ x2)

  rezult = equiv(int(x[0]) or int(x[0]), int(x[1]) and pier(pier(int(x[1]), int(x[0])), int(x[1])))
  return rezult

In [7]:
print("Автомат Мили:\n")
truth_table = truthTable(pos_inputs, pos_states, x, v)
print("\n")
output_table = outputTable_Mili(pos_inputs, pos_states, x, v)

Автомат Мили:

Таблица истинности
+-------+----+----+----+----+
| v \ x | 00 | 01 | 10 | 11 |
+-------+----+----+----+----+
|   00  | 01 | 01 | 10 | 11 |
|   01  | 11 | 01 | 00 | 11 |
|   10  | 01 | 01 | 10 | 10 |
|   11  | 01 | 11 | 10 | 00 |
+-------+----+----+----+----+


Таблица выходов
+-------+------+------+------+------+
| y \ x |  00  |  01  |  10  |  11  |
+-------+------+------+------+------+
|   y   | 1101 | 1101 | 1100 | 1000 |
+-------+------+------+------+------+


In [8]:
#Для удобства и простоты построения матрицы соединений, можно использовать тип DF
truth_copy = pd.DataFrame(truth_table.copy(), columns = pos_states)
#truth_copy['input'] = pd.Series([''.join(pos_inputs[i]) for i in range(len(pos_inputs))], index=truth_copy.index)

out_copy = pd.DataFrame(np.matrix(output_table).copy(), columns = pos_states)
#out_copy['input'] = pd.Series([''.join(pos_inputs[i]) for i in range(len(pos_inputs))], index=out_copy.index)

creatly_connection_table(truth_copy, out_copy)

,00,01,10,11
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0


###Задача 2. Мура

In [9]:
x0_2 = [0, 0]
#2 входа - v
#2 состояния - x
#4 выхода - y

In [10]:
#Выходы и состояния для автомата Мура заданы функциями с логическими операторами

def x1(x, v):
  #v1 -> (v2 & (x2 v v1 v v1 ^ x2 + (v2 -> x1 ~ v2)))

  rezult = implic(int(v[0]), int(v[1]) and (pier(int(x[1]) or int(v[0]) or int(v[0]), int(x[1])) ^ equiv(implic(int(v[1]), int(x[0])), int(v[1]))))
  return rezult

def x2(x, v):
  #x1 | (x2 ^ (v1 ' | (v1 ~ (x1 + (x1 v v2)))))

  rezult = schaeffer(int(x[0]), pier(int(x[1]), schaeffer(not int(v[0]), equiv(int(v[0]), int(x[0]) ^ (int(x[0]) or int(v[1]))))))
  return rezult

def y1(x, v):
  #x1 + v1 -> (x2 ^ v1 | (x1'))

  rezult = implic(int(x[0]) ^ int(v[0]), schaeffer(pier(int(x[1]), int(v[0])), not int(x[0])))
  return rezult

def y2(x, v):
  #x2 v v2' -> (x2~v1)

  rezult = implic(int(x[1]) or (not int(v[1])), equiv(int(x[1]), int(v[0])))
  return rezult

def y3(x, v):
  #x1 & (x1 v (x2 & (x2 ^ (x1 & x2 + x1 | v2 & x1))))

  rezult = int(x[0]) and (int(x[0]) or (int(x[1]) and pier(int(x[1]), (int(x[0]) and int(x[1])) ^ (schaeffer(int(x[0]), int(v[1])) and int(x[0])))))
  return rezult

def y4(x, v):
  #v1 ^ x1' v (x1 v x2 -> x2 & (x2'))

  rezult = pier(int(v[0]), (not int(x[0])) or implic(int(x[0]) or int(x[1]), int(x[1]) and (not int(x[1]))))
  return rezult

In [11]:
print("Автомат Мура:\n")
truth_table2 = truthTable(pos_inputs, pos_states, x, v)
print("\n")
output_table2 = outputTable_Mura(pos_inputs, pos_states, x, v)

Автомат Мура:

Таблица истинности
+-------+----+----+----+----+
| v \ x | 00 | 01 | 10 | 11 |
+-------+----+----+----+----+
|   00  | 11 | 11 | 01 | 01 |
|   01  | 11 | 11 | 01 | 01 |
|   10  | 10 | 10 | 01 | 11 |
|   11  | 11 | 11 | 01 | 11 |
+-------+----+----+----+----+


Таблица выходов
+-------+------+------+------+------+
| v \ x |  00  |  01  |  10  |  11  |
+-------+------+------+------+------+
|   00  | 1100 | 1100 | 1000 | 1100 |
|   01  | 1000 | 1000 | 1100 | 1100 |
|   10  | 1111 | 1111 | 1010 | 1110 |
|   11  | 1011 | 1011 | 1110 | 1110 |
+-------+------+------+------+------+


In [12]:
#Для удобства и простоты построения матрицы соединений, можно использовать тип DF
truth_copy2 = pd.DataFrame(truth_table2.copy(), columns = pos_states)
truth_copy2['input'] = pd.Series([''.join(pos_inputs[i]) for i in range(len(pos_inputs))], index=truth_copy2.index)

out_copy2 = pd.DataFrame(output_table2.copy(), columns = pos_states)
out_copy2['input'] = pd.Series([''.join(pos_inputs[i]) for i in range(len(pos_inputs))], index=out_copy2.index)

creatly_connection_table(truth_copy2, out_copy2)

,00,01,10,11,input
0,0,0,0,0,00
1,0,0,00/1000 v 01/1100 v 10/1010 v 11/1110,00/1100 v 01/1100,01
2,10/1111,10/1111,0,0,10
3,00/1100 v 01/1000 v 11/1011,00/1100 v 01/1000 v 11/1011,0,10/1110 v 11/1110,11


##СКНФ/СДНФ


In [24]:
x = [0, 1]
v = [0, 1]


pos_inputs = []
for i in range(len(v)):
  for j in range(len(v)):
    pos_inputs.append(str(v[i]) + str(v[j]))
print('Возможные входы:', pos_inputs)

pos_states = []
for i in range(len(x)):
  for j in range(len(x)):
    pos_states.append(str(x[i]) + str(x[j]))
print('Возможные состояния:', pos_states)

Возможные входы: ['00', '01', '10', '11']
Возможные состояния: ['00', '01', '10', '11']


###Задача 3. Автомат Мура

In [17]:
x0_3 = [1, 1]
phi = [['11', '01', '10', '10'], ['10', '00', '10', '10'], ['11', '01', '11', '11'], ['11', '01', '10', '10']]
psi = [['1101', '1110', '1100', '1110'], ['1010', '0111', '1111', '0111'], ['1001', '1110', '1101', '1111'], ['1000', '1101', '1101', '1101']]

#2 входа; 2 состояния; 4 выхода

In [19]:
print("Матрица переходов для автомата Мура:\n")
print(np.matrix(phi))
print("\n\nМатрица выходов для автомата Мура:\n")
print(np.matrix(psi))

Матрица переходов для автомата Мура:

[['11' '01' '10' '10']
 ['10' '00' '10' '10']
 ['11' '01' '11' '11']
 ['11' '01' '10' '10']]


Матрица выходов для автомата Мура:

[['1101' '1110' '1100' '1110']
 ['1010' '0111' '1111' '0111']
 ['1001' '1110' '1101' '1111']
 ['1000' '1101' '1101' '1101']]


In [37]:
#поступать на вход будет сама таблица(либо переходов, либо выходов), индекс (номер функции x1,x2 или y1,y2,y3,y4 (у автомата Мили 3 функции у)), pos_states, pos_inputs
#строиться будет для всех комбинаций x1,x2,v1,v2 (функция СКНД или СДНФ из 4 переменных)
#всего будет построено 6 функций скнф и 6 сднф

#СДНФ() (смотреть на 1)
def pdnf(inputTable, index, pos_inputs, pos_states):
  countInputs = len(pos_inputs[0])
  countStates = len(pos_states[0])
  rezultTable = pd.DataFrame(it.product([0,1], repeat=(countInputs + countStates))) #эта таблица отражает возникновение отрицаний в функции СДНФ
  countRows = len(rezultTable)
  function = 'СДНФ = '

  nameCols = []
  for i in range(countStates):
    nameCols.append('x' + str(i + 1))
  for i in range(countInputs):
    nameCols.append('v' + str(i + 1))
  rezultTable.columns = nameCols

  rezultTable['f'] = np.zeros(2**(countInputs + countStates))

  #for i in range(countRows):
    #for j in range(countInputs + countStates):
      #if(inputTable[i,j]):
        #rezultTable['f'] =

  #ЕСТЬ ВАРИК:
  #!!! брать значения x1,x2,v1,v2 и ставить в результирующую таблицу значения из исх. таблицы, которое стоит на пересечении этих переменных
  #внешние 2 цикла для перебора элементов исходной таблицы
  #создать функцию для нахождения индекса строки из результирующей таблице с x1,x2,v1,v2, которые соответствуют входам и состояниям текущего элемента исх.таблицы

  return rezultTable

#СКНФ() (смотреть на 0)
def pcnf():
  combinations = pd.DataFrame(it.product([0,1], repeat=4))

  return 0



In [38]:
pdnf(phi, 0, pos_inputs, pos_states)

16


,x1,x2,v1,v2,f
0,0,0,0,0,0.0
1,0,0,0,1,0.0
2,0,0,1,0,0.0
3,0,0,1,1,0.0
4,0,1,0,0,0.0
5,0,1,0,1,0.0
6,0,1,1,0,0.0
7,0,1,1,1,0.0
8,1,0,0,0,0.0
9,1,0,0,1,0.0


###Задача 4. Автомат Мили

In [22]:
x0_4 = [0, 1]
phi_2 = [['00', '00', '10', '10'], ['10', '10', '10', '10'], ['01', '01', '11', '11'], ['00', '00', '10', '10']]
psi_2 = [['011', '010', '101', '011']]

#2 входа; 2 состояния; 3 выхода

In [23]:
print("Матрица переходов для автомата Мили:\n")
print(np.matrix(phi_2))
print("\n\nМатрица выходов для автомата Мили:\n")
print(np.matrix(psi_2))

Матрица переходов для автомата Мили:

[['00' '00' '10' '10']
 ['10' '10' '10' '10']
 ['01' '01' '11' '11']
 ['00' '00' '10' '10']]


Матрица выходов для автомата Мили:

[['011' '010' '101' '011']]
